In [2]:
from pdfminer.high_level import extract_text
import os
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import faiss
import numpy as np
import torch

torch.manual_seed(42)
model_encoder = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\wisj9\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\wisj9\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


## PDF 처리

In [3]:
pdfs = [f"datas/건설안전지침/{i}" for i in os.listdir("datas/건설안전지침")]
# print("\n".join(pdfs))
print(f"PDF 총 개수: {len(pdfs)}")

whole = ""
for i in range(len(pdfs)):
    pdf = extract_text(pdfs[i])

    pdf = pdf.split("\n5. ")
    if len(pdf) >= 3: 
        pdf = pdf[2:]
    else: pdf = pdf[1:]
    pdf = " ".join(pdf)
    whole += pdf

elements = re.findall(r'\(([가-하1-9])\)', whole)
whole = re.split(r'\(([가-하1-9])\)', whole)
print(len(whole))

PDF 총 개수: 104
15075


In [4]:
whole = [_ for _ in whole if len(_) != 1]
whole = list(map(lambda x: x.replace("\n", " "), whole))
whole = list(map(lambda x: x.replace("KOSHA GUIDE", ""), whole))
for i in whole:
    print(i)

공통적인 안전조치사항  
 관리감독자는 작업시작 전, 근로자에게 안전 작업순서, 방법, 절차 등을 숙  지시켜야 한다.  
 시공자는 고소작업에 따른 위험요인에 대한 근로자들의 안전을 고려하여  추락 및 낙하물 방지시설을 설치하여야 한다.  
 추락의 위험이 있는 작업발판에는 근로자가 안전하게 승강할 수 있는 승강  설비 및 안전난간을 갖추어야 한다.  
 관리감독자는 작업계획을 수립하고 작업시작 전 또는 작업 중 다음 사항을  준수하여야 한다.  
 재료 ᠊  기구의 결함 유무를 점검하고 불량품을 제거 
 올바른 작업 방법 및 순서를 근로자들에게 교육  
 작업방법을 지휘하고 이를 감시  - 7 -    C - 67 - 2016  
 근로자의 보호구 착용상태를 감시  
 작업전 관리감독자는 위험성평가를 실시하여 유해․위험요소를 확인하고  작업 중에도 관리감독을 철저히 하여 재해예방을 하여야 한다.  
 주두부 및 측경간부 시공시 동바리 설치방법, 콘크리트 타설방법, 시공순서  등에 대해 세밀하게 검토하여 안전 작업계획을 수립해야 한다.  
 화재의 위험이 있는 용접 및 용단 작업장소에는 소화기, 방화수 등을 비치  하여 초기 소화할 수 있도록 하여야 한다.  
 크레인 작업시에는  C-48-2012 (건설기계 안전보건작업 지  침)과  C-99-2015(이동식 크레인 양중작업의 안정성 검토  지침),  C-102-2014[건설현장의 중량물 취급 작업계획서(이  동식 크레인) 작성지침]를 준수하여야 하고, 근로자에게 이를 교육시켜야  한다.  
 강풍 발생시 자재 등의 낙하 및 비래 예방조치를 하여야 한다.  (10) 작업 후 작업장 및 통로등의 정리정돈을 실시하여야 한다.  - 8 -    C - 67 - 2016  6. 주두부 안전작업  주두부 공사의 전경은 <그림 5>와 같으며 시공시 유의사항은 다음과 같다.  <그림 5> 교각위 주두부 설치사진  
 주두부 콘크리트에 프리스트레스를 도입할 경우 거푸집 및 동바리는 콘크  리트의 탄성변형을 구속하지 않

In [48]:
train = pd.read_csv("datas/train.csv")
test = pd.read_csv("datas/test.csv")
print(len(train), len(test))

23422 964


In [49]:
train.iloc[0]
test.iloc[0]

ID                                                  TEST_000
발생일시                                     2024-06-03 오전 09:39
사고인지 시간                                               정규작업 -
날씨                                                        맑음
기온                                                       27℃
습도                                                       53%
공사종류                                    건축 / 건축물 / 교정 및 군사시설
연면적                                                1,990.32㎡
층 정보                                            지상 1층, 지하 0층
인적사고                                                     부딪힘
물적사고                                                      전도
공종                                             건축 > 철근콘크리트공사
사고객체                                           건설기계 > 콘크리트펌프
작업프로세스                                                  타설작업
장소                                            교정 및 군사시설 / 외부
부위                                               콘크리트펌프 / 바닥
사고원인       펌프카 아웃트리거 바닥 

In [50]:
train["for_rag"] = train.apply(lambda row: f"""발생 일시: {row["발생일시"][:4]+"년"+row["발생일시"][5:7]+"월"+row["발생일시"][8:10]+"일"}({row["사고인지 시간"]})
날씨: {row["날씨"]}
기온/습도: {row["기온"]}/{row["습도"]}
공사 종류: {row["공사종류"]} / {row["공종"]} / {row["작업프로세스"]}
사고 내용(인명 사고): {row["인적사고"]}
사고 내용(재산 피해): {row["물적사고"]}
사고 발생 장소: {row["장소"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

test["for_rag"] = test.apply(lambda row: f"""발생 일시: {row["발생일시"][:4]+"년"+row["발생일시"][5:7]+"월"+row["발생일시"][8:10]+"일"}({row["사고인지 시간"]})
날씨: {row["날씨"]}
기온/습도: {row["기온"]}/{row["습도"]}
공사 종류: {row["공사종류"]} / {row["공종"]} / {row["작업프로세스"]}
사고 내용(인명 사고): {row["인적사고"]}
사고 내용(재산 피해): {row["물적사고"]}
사고 발생 장소: {row["장소"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

train["for_rag_weather"] = train.apply(lambda row: f"""발생 일시: {row["발생일시"][:4]+"년"+row["발생일시"][5:7]+"월"+row["발생일시"][8:10]+"일"}({row["사고인지 시간"]})
날씨: {row["날씨"]}
기온/습도: {row["기온"]}/{row["습도"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

test["for_rag_weather"] = test.apply(lambda row: f"""발생 일시: {row["발생일시"][:4]+"년"+row["발생일시"][5:7]+"월"+row["발생일시"][8:10]+"일"}({row["사고인지 시간"]})
날씨: {row["날씨"]}
기온/습도: {row["기온"]}/{row["습도"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

train["for_rag_accident"] = train.apply(lambda row: f"""
공사 종류: {row["공사종류"]} / {row["공종"]} / {row["작업프로세스"]}
사고 내용(인명 사고): {row["인적사고"]}
사고 내용(재산 피해): {row["물적사고"]}
사고 발생 장소: {row["장소"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

test["for_rag_accident"] = test.apply(lambda row: f"""
사고 내용(인명 사고): {row["인적사고"]}
사고 내용(재산 피해): {row["물적사고"]}
사고 원인: {row["사고원인"]}
""",axis = 1)

try:
    doc_embeddings_weather = np.load("doc_embeddings_weather.npy")
except:
    doc_embeddings_weather = model_encoder.encode(list(train["for_rag_weather"]), convert_to_numpy=True)
    np.save("doc_embeddings_weather.npy", doc_embeddings_weather)

dim_weather = doc_embeddings_weather.shape[1]
index_weather = faiss.IndexFlatL2(dim_weather)
index_weather.add(doc_embeddings_weather)

try:
    doc_embeddings_accident = np.load("doc_embeddings_accident.npy")
except:
    doc_embeddings_accident = model_encoder.encode(list(train["for_rag_accident"]), convert_to_numpy=True)
    np.save("doc_embeddings_accident.npy", doc_embeddings_accident)

dim_accident = doc_embeddings_accident.shape[1]
index_accident = faiss.IndexFlatL2(dim_accident)
index_accident.add(doc_embeddings_accident)

try:
    doc_embeddings = np.load("doc_embeddings.npy")
except:
    doc_embeddings = model_encoder.encode(list(train["for_rag"]), convert_to_numpy=True)
    np.save("doc_embeddings.npy", doc_embeddings)

dim = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(doc_embeddings)

In [51]:
print("원문 쿼리")
query = test.iloc[0]["for_rag"]
query_weather = test.iloc[0]["for_rag_weather"]
query_accident = test.iloc[0]["for_rag_accident"]
print(query+"\n---------------------------------------------------------------------------------")
query_embedding = model_encoder.encode([query], convert_to_numpy=True)
query_embedding_weather = model_encoder.encode([query_weather], convert_to_numpy=True)
query_embedding_accident = model_encoder.encode([query_accident], convert_to_numpy=True)

# 가장 유사한 top_k 문서 검색
top_k = 1
distances, indices = index.search(query_embedding, top_k)
distances_weather, indices_weather = index_weather.search(query_embedding_weather, top_k)
distances_accident, indices_accident = index_accident.search(query_embedding_accident, top_k)

# 검색된 문서 출력
retrieved_docs = [list(train["for_rag"])[i] +"\n재발방지대책 및 향후조치계획: "+ list(train["재발방지대책 및 향후조치계획"])[i]+ "\n\n" for i in indices[0]]
retrieved_docs_weather = [list(train["for_rag"])[i]+"\n재발방지대책 및 향후조치계획: " + list(train["재발방지대책 및 향후조치계획"])[i]+ "\n\n" for i in indices_weather[0]]
retrieved_docs_accident = [list(train["for_rag"])[i]+"\n재발방지대책 및 향후조치계획: " + list(train["재발방지대책 및 향후조치계획"])[i]+ "\n\n" for i in indices_accident[0]]

retrieved_docs += retrieved_docs_weather
retrieved_docs += retrieved_docs_accident

rag = ""
for i in range(len(retrieved_docs)):
    rag += f"example {i}:\n"+retrieved_docs[i]
print(rag)

원문 쿼리
발생 일시: 2024년06월03일(정규작업 -)
날씨: 맑음
기온/습도: 27℃/53%
공사 종류: 건축 / 건축물 / 교정 및 군사시설 / 건축 > 철근콘크리트공사 / 타설작업
사고 내용(인명 사고): 부딪힘
사고 내용(재산 피해): 전도
사고 발생 장소: 교정 및 군사시설 / 외부
사고 원인: 펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨

---------------------------------------------------------------------------------
example 0:
발생 일시: 2023년03월01일(정규작업 -)
날씨: 맑음
기온/습도: 2℃/40%
공사 종류: 토목 / 하천 / 기타 / 토목 > 해체 및 철거공사 / 해체작업
사고 내용(인명 사고): 부딪힘
사고 내용(재산 피해): 전도
사고 발생 장소: 기타 / 외부
사고 원인: 안동소수력 건설현장에서 지반차수공사(CIP)을 위한 부지정지 포크레인 작업 중 측면 전도가 발생한 사고임. 평탄지역으로 이동한 상태에서 가이드블럭 이동을 하여야 했으나  비탈경사면에서 장비를 스윙하여 측면으로 전도된 사고임.

재발방지대책 및 향후조치계획: 휴일근무제 운영 강화 및 자제, 운전자 안전교육 및 재교육 강화, 장비 운전자 및 작업자 교육 강화, 작업 절차 숙지, 장비 운전자 채용 시 정비상태 및 숙련도 검토 강화, 장비 접근금지 스티커 부착, 작업장 내 안전 확보를 통한 사고 예방 조치.

example 1:
발생 일시: 2023년03월01일(정규작업 -)
날씨: 맑음
기온/습도: 2℃/40%
공사 종류: 토목 / 하천 / 기타 / 토목 > 해체 및 철거공사 / 해체작업
사고 내용(인명 사고): 부딪힘
사

In [52]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-14B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 8/8 [00:09<00:00,  1.19s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [53]:
prompt = query

messages = [
    {"role": "system", 
     "content": "너는 주어진 사고 case를 보고, 이에 대한 방지 대책을 한 문장으로 정리하는 전문가야."},
    {"role": "examples",
    "content": rag},
    {"role": "user",
     "content": prompt +"재발방지대책 및 향후조치계획: "}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    do_sample=False,             # 샘플링 OFF
    num_beams=1,                 # Greedy decoding (Beam search 안 씀)
    temperature=1.0,
    top_k=50,
    top_p=1.0
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

타설 작업 시 펌프카의 안전성을 확보하기 위해 지반 상태와 장비의 무게중심을 정밀히 분석하고, 필요한 경우 추가 보강 및 조정을 실시하며, 작업 환경에 맞는 적절한 장비 사용을 강제해야 한다.


In [55]:
def get_answer(row):
    print("answering-"+ row["ID"])
    query = row["for_rag"]
    query_weather = row["for_rag_weather"]
    query_accident = row["for_rag_accident"]
    query_embedding = model_encoder.encode([query], convert_to_numpy=True)
    query_embedding_weather = model_encoder.encode([query_weather], convert_to_numpy=True)
    query_embedding_accident = model_encoder.encode([query_accident], convert_to_numpy=True)

    # 가장 유사한 top_k 문서 검색
    top_k = 2
    distances, indices = index.search(query_embedding, top_k)
    distances_weather, indices_weather = index_weather.search(query_embedding_weather, top_k)
    distances_accident, indices_accident = index_accident.search(query_embedding_accident, top_k)

    # 검색된 문서 출력
    retrieved_docs = [list(train["for_rag"])[i] +"\n재발방지대책 및 향후조치계획: "+ list(train["재발방지대책 및 향후조치계획"])[i]+ "\n\n" for i in indices[0]]
    retrieved_docs_weather = [list(train["for_rag"])[i]+"\n재발방지대책 및 향후조치계획: " + list(train["재발방지대책 및 향후조치계획"])[i]+ "\n\n" for i in indices_weather[0]]
    retrieved_docs_accident = [list(train["for_rag"])[i]+"\n재발방지대책 및 향후조치계획: " + list(train["재발방지대책 및 향후조치계획"])[i]+ "\n\n" for i in indices_accident[0]]

    retrieved_docs += retrieved_docs_weather
    retrieved_docs += retrieved_docs_accident

    rag = ""
    for i in range(len(retrieved_docs)):
        rag += f"example {i}:\n"+retrieved_docs[i]

    messages = [
        {"role": "system", 
        "content": "너는 주어진 사고 case를 보고, 이에 대한 방지 대책을 한 문장으로 정리하는 전문가야.\n"+rag},
        {"role": "user",
        "content": query +"재발방지대책 및 향후조치계획: "}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,
        do_sample=False,
        num_beams=1, 
        temperature=1.0,
        top_k=50,
        top_p=1.0
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)
    return response

test["answer"] = test.apply(lambda row: get_answer(row), axis =1)


answering-TEST_000
펌프카 아웃트리거의 안전한 설치와 균형 유지, 지반 상태 점검 강화 및 타설 작업 시 무게중심 조절을 위한 교육 실시를 통해 재발 방지 대책을 마련한다.
answering-TEST_001
작업자 안전교육 강화와 보안면 착용 의무화, 그리고 숫돌 사용 시 안전 점검 강화를 통해 재발 방지.
answering-TEST_002
작업 이동 시 전방주시와 안전수칙 준수를 위한 근로자 안전교육 실시 및 작업 전 안전점검 강화.
answering-TEST_003
작업 후 발판 및 작업면의 잔여물 철저한 정리와 안전교육 강화를 통해 재발 방지.
answering-TEST_004
점심시간 이동 시에도 안전의식을 고취하고, 작업 전후 안전교육을 강화하여 동일한 사고의 재발을 방지한다.
answering-TEST_005
안전발판 설치 점검 강화와 작업절차 준수 교육을 통해 재발 방지 대책을 마련한다.
answering-TEST_006
작업자의 안전의식 향상을 위한 안전교육 강화와 현장 관리 감독을 강화하는 조치.
answering-TEST_007
SPLICE PLATE 설치 작업 시 안전 거리를 확보하고 작업자 위치를 체크하는 절차를 마련하여 재발 방지.
answering-TEST_008
이동식 틀비계 사용 시 하단 위치 확인을 위한 안전 점검 체계 구축 및 균형 잡히지 않은 상태에서의 이동 금지 안내를 통한 재발 방지.
answering-TEST_009
작업 중 철골 구조물과의 안전거리 유지와 정밀한 작업 점검을 통한 재발 방지 대책 마련.
answering-TEST_010
고령 작업자의 건강 상태와 능력을 수시로 점검하고, 필요시 작업 부담 경감 및 적절한 지원을 제공하여 재발 방지.
answering-TEST_011
장비 이동 궤도의 안전성 확보와 정기 점검을 통해 재발 방지 대책을 마련한다.
answering-TEST_012
재발방지대책 및 향후조치계획: 합판 자재 운반 시 안전한 방법 교육 및 2인 1조 작업 실시.
answering-T

KeyboardInterrupt: 

In [ ]:
test["answer"] = test["answer"].apply(lambda x: x.split("재발방지대책 및 향후조치계획: ")[1] if len(x.split("재발방지대책 및 향후조치계획: ")) == 2 else x)
test["answer"] = test["answer"].apply(lambda x: x.split("재발방지대책 및 향후조치계획:")[1] if len(x.split("재발방지대책 및 향후조치계획:")) == 2 else x)
test["answer"] = test["answer"].apply(lambda x: x.split("재발방지대책 및 �향후조치계획:")[1] if len(x.split("재발방지대책 및 �향후조치계획:")) == 2 else x)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = model_encoder.encode(test["answer"])
print(pred_embeddings.shape)  # (샘플 개수, 768)

submission = pd.read_csv('datas/sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
submission.iloc[:,1] = test["answer"]
submission.iloc[:,2:] = pred_embeddings
submission.head()
submission.to_csv("sub.csv", index = False)

(964, 768)
